In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance, distance_matrix
from scipy.spatial.distance import pdist
from pandas import ExcelWriter
from pandas import ExcelFile
import math

from typing import Tuple

In [2]:
X_8 = pd.read_excel('8clusters.xlsx', sheetname='Data')
X_12 = pd.read_excel('12clusters.xlsx', sheetname='Data')
X_14 = pd.read_excel('14clusters.xlsx', sheetname='Data')
X_14

ID  C14  ATS0  ATD0  Pulse0  ATS1  ATD1  Pulse1  ATS2  ATD2  Pulse2  \
0        1    3   120    75      66   148    83     106   131    75      79   
1        2    7   109    62      95   124    74     108   117    68     105   
2        3    1   130    85      82   175    89     102   149    86      84   
3        4    2   130    78      89   153    94     122   152    87      98   
4        5    1   133    73      76   138    71      98   129    78      92   
5        6    9   130    79      53   150    74      72   162    76      56   
6        7    9   140    81      51   162    80      67   151    92      51   
7        8    4   138    99      86   143    93     121   167   105     102   
8        9    3   132    79     103   136    83     108   127    76      86   
9       10    9   106    70      57   144    84      64   145    69      66   
10      11    2   114    65      93   113    73     117   116    70     109   
11      12    5   142    80      64   165    66      81   150    75      69   
12      13   13   106    63      68   130    73     101   117    69      70   
13      14    4   138    91     112   175    91     118   136    88     103   
14      15   12   103    63      69   112    62     102   123    61      87   
15      16    3   129    78      68   136    73      95   124    73      71   
16      17    8   132    85     104   137    76     115   123    74     104   
17      18   13   114    70      56   120    67      87   118    73      59   
18      19    7   116    80      88   134    83     112   118    76      93   
19      20    5   151    93      78   178    87      82   149    85      72   
20      21    1   144    80      75   165   129      85   138    77      80   
21      22   13   104    70      67   123    70      99   120    74      84   
22      23    3   126    78      79   140    77      97   138    73      79   
23      24    3   122    81      82   131    77      89   134    79      77   
24      25    2   111    69      87   133    78     124   119    74     104   
25      26    3   114    78      70   132    78      74   118    87      65   
26      27    4   146    84      96   159    95     110   144    92     108   
27      28    3   138    76      87   143    83     100   131    71      88   
28      29    6   124    78     101   138    93     122   152    85     108   
29      30    2   133    77      96   133    78     119   134    70     103   
...    ...  ...   ...   ...     ...   ...   ...     ...   ...   ...     ...   
1077  1078    1   149    74      69   139    70     109   149    69      93   
1078  1079    6   138    78     102   157    78     119   154    81     111   
1079  1080    2   122    62      98   141    71     118   111    66      99   
1080  1081    8   127    69     112   133    75     130   128    73     119   
1081  1082    2   129    70     103   128    62     132   134    75     111   
1082  1083   10   128    62      64   125    63      87   120    59      78   
1083  1084   10   117    62      59   135    64      86   139    62      57   
1084  1085    2   137    78      94   134    82     107   136    79      93   
1085  1086    2   123    74      77   120    71     114   119    78     105   
1086  1087   10   125    67      63   117    59     110   115    66      87   
1087  1088   10   131    62      58   148    69      95   134    65      81   
1088  1089   10   125    62      64   125    69     100   128    70      75   
1089  1090    9   118    73      60   159    77      92   146    75      74   
1090  1091    9   136    65      60   173    70      77   148    64      65   
1091  1092    4   158    97      96   143    85     125   154    90     106   
1092  1093    4   115    90      95   120    82     112   144   105     105   
1093  1094    5   146    85      79   161    86     109   168    82      83   
1094  1095   13   109    62      63    98    57      98   104    60      72   
1095  1097    4   124    94      84   138    91     110   

In [3]:
def centrs_cluster(X):
    data = X.iloc[:,2:]
    return [data.loc[X.iloc[:,1] == i].mean().as_matrix() for i in range(1, len(X.iloc[:,1].unique())+1)]

def differences(X, m):
    list_ = list()
    for index, u in X.iterrows():
        list_.append(distance.euclidean(u,m))
    return list_

def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)
    
def up(X):
    clusters = X.iloc[:,1].value_counts().index.tolist()
    num_objects = X.iloc[:,1].value_counts()
    centres = centrs_cluster(X)
    data = X.iloc[:,2:] 
    res = []
    for c in clusters:
        centre = centres[c-1]
        diff = sum(differences(data.loc[X.iloc[:,1] == c], centre))/num_objects[c]
        res.append(diff)
    return sum(res)/len(res)

def down(X):
    clusters = X.iloc[:,1].value_counts().index.tolist()
    num_objects = X.iloc[:,1].value_counts()
    centres = centrs_cluster(X)
    data = X.iloc[:,2:] 
    res = []
    return sum(pdist(centrs_cluster(X)))/nCr(len(clusters), 2)

In [9]:
up_8 = up(X_8)
up_12 = up(X_12)
up_14 = up(X_14)


In [10]:
down_8 = down(X_8)
down_12 = down(X_12)
down_14 = down(X_14)

In [14]:
pd.DataFrame([[up_8, up_12, up_14],
              [down_8, down_12, down_14],
              [up_8/down_8, up_12/down_12, up_14/down_14]
             ]
            )

0          1          2
0  36.842863  28.537577  31.603595
1  66.920037  76.606897  77.296405
2   0.550551   0.372520   0.408862